In [3]:
#install.packages("lubridate", repos="http://cran.r-project.org")
# api key sWyMx5Thd3sPhF613nE6
library(Quandl)
library(RCurl)
library(httr)
library(stats)
library(zoo)
library(lubridate)
set_config( config( ssl_verifypeer = 0L ) )
Quandl.api_key('sWyMx5Thd3sPhF613nE6')

In [4]:
trade_econ_data <- function(country, code, excode = 'NULL', ycode,
                            monthly, quarterly, col_names ){
    
    data_list <- list()
    
    # Monthly Data
    for (data_set in monthly){
        print(paste('Downloading',data_set,sep=' '))
        link <- paste('SGE/',code,data_set,sep='')
        df <- Quandl(link, start_date = '1971-01-01', collapse = 'monthly')
        colnames(df) <- c('Date',data_set)
        df$Date <- round_date(df$Date, unit = 'month')
        data_list[[length(data_list)+1]] <- df
        
    }
    
    
    
    # Quarterly Data
    for (data_set in quarterly){
        print(paste('Downloading',data_set,sep=' '))
        link <- paste('SGE/',code,data_set,sep='')
        df <- Quandl(link, start_date = '1971-01-01', collapse = 'quarterly')
        colnames(df) <- c('Date',data_set)
        df$Date <- round_date(df$Date, unit = 'month')
        data_list[[length(data_list)+1]] <- df
    }
    
    # Government Yield
    if (ycode != 'NULL'){
        print(paste('Downloading','yield',sep=' '))
        link <- paste('YC/',ycode,sep='')
        df <- Quandl(link, start_date = '1971-01-01', collapse = 'monthly')
        colnames(df) <- c('Date','3M_Yield')
        df$Date <- round_date(df$Date, unit = 'month')
        data_list[[length(data_list)+1]] <- df
    }
    
    # Exchange Rate
    if (excode != 'NULL'){
        print(paste('Downloading','exchange rate',sep=' '))
        link <- paste('FRED/',excode,sep='')
        df <- Quandl(link, start_date = '1971-01-01', collapse = 'monthly')
        colnames(df) <- c('Date','Exchange')
        df$Date <- round_date(df$Date, unit = 'month')
        data_list[[length(data_list)+1]] <- df
    }

    
    # Merge all of the data into one date frame by date
    df <- Reduce(function(...) merge(..., by='Date', all=TRUE), data_list)
    df$Country <- rep(country,dim(df)[1])
    colnames(df) <- col_names
    
    
    # Interpolate intermediate values for quarterly data using splines
    start <- length(monthly)+2
    end <- start + length(quarterly)
    for( i in start:end){
        if( sum(is.na(df[,i])) > 0 ){
            df[,i] <- na.spline(df[,i])
        }
    }
    
    


    filename <- paste(country,'.csv',sep='')
    setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Data')
    write.csv(df, file = filename)
    return(df)
}

In [5]:
# USA Data Frame
usa <- trade_econ_data(country = 'usa', code = 'USA', excode = 'NULL', ycode = 'USA1Y',
                            monthly = c('CPIC','BOT','GYLD','FER','IR','BLR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_d','BOT_d',
                                  'Yield_d','FER_d','Int_d','PrimeRate_d','1Y_Yield_d',
                                  'GDPG_d','CA_d','FDI_d','Country'))
tail(usa)

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading BLR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading yield"


,Date,Infl_d,BOT_d,Yield_d,FER_d,Int_d,PrimeRate_d,1Y_Yield_d,GDPG_d,CA_d,FDI_d,Country
537,2015-10-01,0,-42484,2.0368,120218,0.25,3.25,2,-124120,38492,0.33,usa
538,2015-11-01,0.2,-44610,2.1421,119267,0.25,3.25,1.088602,-276419.5,33914.59,0.34,usa
539,2015-12-01,0.5,-42226,2.206,116640,0.25,3.25,0.5836637,-507177.9,25925.97,0.51,usa
540,2016-01-01,0.7,-43357,2.2694,117581,0.5,3.37,0.7,-829082.1,13871.86,0.65,usa
541,2016-02-01,1.4,NA,1.9209,NA,0.5,3.5,1.652426,-1254819,-2902.051,0.47,usa
542,2016-03-01,NA,NA,1.7449,NA,NA,NA,3.655755,-1797074,-25050.05,0.53,usa


In [6]:
# Euro-zone Data Frame
# No GYLD or BLR data
euro <- trade_econ_data(country = 'europe', code = 'EUR', excode = 'EXUSEU', 
                        ycode = 'DEU1Y',
                            monthly = c('CPIC','BOT','FER','IR'),
                            quarterly = c('GGR','CA'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'FER_f','Int_f','1Y_Yield_f','GDPG_f',
                                  'CA_f','Exchange','Country'))
euro$Exchange <- 1/euro$Exchange
setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Data')
write.csv(euro, file = 'europe.csv')
tail(euro)

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading yield"
[1] "Downloading exchange rate"


,Date,Infl_f,BOT_f,FER_f,Int_f,1Y_Yield_f,GDPG_f,CA_f,Exchange,Country
297,2015-10-01,-0.1,19950.7,332253,0.05,0.3,33.8,-0.27,0.8905513,europe
298,2015-11-01,0.1,23989.8,324595,0.05,0.2854791,33.8854,-0.34,0.8906306,europe
299,2015-12-01,0.2,23716.5,323777,0.05,0.2842445,36.16762,-0.41,0.9322271,europe
300,2016-01-01,0.2,24281.7,333872,0.05,0.3,41.4,-0.39,0.918358,europe
301,2016-02-01,0.4,NA,NA,0.05,0.3364493,50.33588,-0.46,0.9212345,europe
302,2016-03-01,NA,NA,NA,NA,0.3972961,63.72858,-0.5,0.8971024,europe


In [7]:
# Australia Data Frame
# No BLR data, No 1Y yield
aus <- trade_econ_data(country = 'australia', code = 'AUS', excode = 'EXUSAL',
                       ycode = 'NULL',
                            monthly = c('CPIC','BOT','GYLD','FER','IR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f',                                  'GDPG_f',
                                  'CA_f','FDI_f','Exchange','Country'))
aus$Exchange <- 1/aus$Exchange
setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Data')
write.csv(euro, file = 'australia.csv')
tail(aus)

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading exchange rate"


,Date,Infl_f,BOT_f,Yield_f,FER_f,Int_f,GDPG_f,CA_f,FDI_f,Exchange,Country
537,2015-10-01,1.5,-2063,2.605,72670,2,0.9,-18104,56027.78,1.416631,australia
538,2015-11-01,NA,-2317,2.611,64337,2,1.795153,-12805.35,55703.4,1.388889,australia
539,2015-12-01,NA,-2727,2.86,68208,2,3.173684,-4445.988,55340.94,1.399384,australia
540,2016-01-01,1.7,-3535,2.88,67440,2,5.117076,7461.133,54938.62,1.379881,australia
541,2016-02-01,NA,NA,2.635,61585,2,7.70681,23403.05,54494.69,1.42633,australia
542,2016-03-01,NA,NA,2.43,NA,2,11.02437,43866.79,54007.38,1.407856,australia


In [8]:
# Canada Data Frame
can <- trade_econ_data(country = 'canada', code = 'CAN', excode = 'EXCAUS',
                       ycode = 'CAN1Y',
                            monthly = c('CPIC','BOT','GYLD','FER','IR','BLR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','PrimeRate_f','1Y_Yield_f',
                                  'GDPG_f','CA_f','FDI_f','Exchange','Country'))

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading BLR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading yield"
[1] "Downloading exchange rate"


In [9]:
# Denmark Data Frame
# No BLR, FDI data, No 1Y yield
den <- trade_econ_data(country = 'denmark', code = 'DNK', excode = 'EXDNUS',
                       ycode = 'NULL',
                            monthly = c('CPIC','BOT','GYLD','FER','IR'),
                            quarterly = c('GGR','CA'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f',
                                  'GDPG_f','CA_f','Exchange','Country'))

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading exchange rate"


In [10]:
# Japan Data Frame
jap <- trade_econ_data(country = 'japan', code = 'JPN', excode = 'EXJPUS', 
                       ycode = 'JPN1Y',
                            monthly = c('CPIC','BOT','GYLD','FER','IR','BLR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','PrimeRate_f','1Y_Yield_f',
                                  'GDPG_f','CA_f','FDI_f','Exchange','Country'))

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading BLR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading yield"
[1] "Downloading exchange rate"


In [11]:
# Korea Data Frame
korea <- trade_econ_data(country = 'korea', code = 'KOR', excode = 'EXKOUS',
                         ycode = 'KOR1Y',
                            monthly = c('CPIC','BOT','GYLD','FER','IR','BLR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','PrimeRate_f','1Y_Yield_f',
                                  'GDPG_f','CA_f','FDI_f','Exchange','Country'))

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading BLR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading yield"
[1] "Downloading exchange rate"


In [12]:
# Mexico Data Frame
# No BLR data
mex <- trade_econ_data(country = 'mexico', code = 'MEX', excode = 'EXMXUS',
                       ycode = 'MEX1Y',
                            monthly = c('CPIC','BOT','GYLD','FER','IR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','1Y_Yield_f',
                                  'GDPG_f','CA_f','FDI_f','Exchange','Country'))

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading yield"
[1] "Downloading exchange rate"


In [13]:
# New Zealand Data Frame
nzl <- trade_econ_data(country = 'new_zealand', code = 'NZL', excode = 'EXUSNZ',
                       ycode = 'NZL1Y',
                            monthly = c('CPIC','BOT','GYLD','FER','IR','BLR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','PrimeRate_f','1Y_Yield_f',
                                  'GDPG_f','CA_f','FDI_f','Exchange','Country'))
nzl$Exchange <- 1/nzl$Exchange
setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Data')
write.csv(nzl, file = 'new_zealand.csv')

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading BLR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading yield"
[1] "Downloading exchange rate"


In [14]:
# Norway Data Frame
# No BLR or FDI data
nor <- trade_econ_data(country = 'norway', code = 'NOR', excode = 'EXNOUS',
                       ycode = 'NOR12M',
                            monthly = c('CPIC','BOT','GYLD','FER','IR'),
                            quarterly = c('GGR','CA'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','1Y_Yield_f',
                                  'GDPG_f','CA_f','Exchange','Country'))

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading yield"
[1] "Downloading exchange rate"


In [15]:
# Sweden Data Frame
# No BLR data
swe <- trade_econ_data(country = 'sweden', code = 'SWE', excode = 'EXSZUS',
                       ycode = 'SWE12M',
                            monthly = c('CPIC','BOT','GYLD','FER','IR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','1Y_Yield_f',
                                  'GDPG_f','CA_f','FDI_f','Exchange','Country'))

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading yield"
[1] "Downloading exchange rate"


In [16]:
# Switzerland Data Frame
# No BLR data
switz <- trade_econ_data(country = 'switzerland', code = 'CHE', excode = 'EXSZUS',
                         ycode = 'CHE12M',
                            monthly = c('CPIC','BOT','GYLD','FER','IR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','1Y_Yield_f',
                                  'GDPG_f','CA_f','FDI_f','Exchange','Country'))

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
[1] "Downloading yield"
[1] "Downloading exchange rate"


In [17]:
# United Kingdom Data Frame
uk <- trade_econ_data(country = 'uk', code = 'GBR', excode = 'NULL',
                      ycode = 'NULL',
                            monthly = c('CPIC','BOT','GYLD','FER','IR','BLR'),
                            quarterly = c('GGR','CA','FDI'), 
                    col_names = c('Date','Infl_f','BOT_f',
                                  'Yield_f','FER_f','Int_f','PrimeRate_f',
                                  'GDPG_f','CA_f','FDI_f','Country'))
exchange <- Quandl("FED/RXI_US_N_M_UK", start_date = '1971-01-01')
uk$Exchange <- 1/exchange$Value
setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Data')
write.csv(uk, file = 'uk.csv')

[1] "Downloading CPIC"
[1] "Downloading BOT"
[1] "Downloading GYLD"
[1] "Downloading FER"
[1] "Downloading IR"
[1] "Downloading BLR"
[1] "Downloading GGR"
[1] "Downloading CA"
[1] "Downloading FDI"
